## ДЗ 1. Сбор текстовых данных (Трусковская Д.Р. МФР232)
#### _Парсинг комментариев о TON с сайта bitcointalk.org_

**_1. Почему TON?_** \
Мне кажется это довольно интересная сейчас криптовалюта в СНГ-сегменте рынка (да и не только). Я выбрала для анализа валюту TON, потому что я думаю что она довольно интересная сейчас и на нее сентимент может иметь сильное влияние. Она связана с известными предпринимателями Павлом Дуровым и Николаем Дуровом и их известным месенджером Telegram, который является одной из крупнейших мессенджеров в СНГ-сегменте и даже мире. Инфраструктуру TON также могут использовать валюты HMSTR и NOT и другие, что также может влиять на курс TON. \
**Дневные котировки [Toncoin](https://ru.investing.com/crypto/toncoin/historical-data) скачены с Investing за период с 20.09.2023 до 04.11.2024**

**_2. Почему bitcointalk.org?_** \
TON не такая популярная валюта, как BTC и ETH, и не является одной из популярных мем-койнов, поэтому кроме телеграмм-каналов сложно найти сайт, на котором было бы именно активное обсуждение TON. Я подумала что на древнем форуме, посвященном всем криптовалютам должно быть достаточное обсуждение и TON (однако и там, если выбираеть только по теме обсуждения не очень много комментариев) \

На данном сайте в общем хабе Обсуждений альткойнов отображается дата последнего комментария к каждому обсуждению. На этом основании и происходит ранжирование обсуждений (обсуждение с самым последним комментарием отображается первым, исключение - несколько закрепленных тем) \

Каждое обсуждение и каждый комментарий к нему имеет собственные дату и время \
**Сайт [Bitcointalk](https://bitcointalk.org/) у меня не открывается без VPN**

In [1]:
import warnings
warnings.filterwarnings("ignore")

**Концепция:** 
1. Собрать все ссылки за 1 год 1 месяц с [Обсуждения альткойнов на BT](https://bitcointalk.org/index.php?board=67.0). Я решила взять с запасом, не 12, а 13 месяцев
2. Парсить все страницы и добавлять в общий df
3. Применить фильтр по ключевым словам

Также я парсила в основном все по XPath

### Пример парсинга одной страницы сайта

Я использую метод undetected_chromedriver, так как он помогает обойти капчу Cloudflare, которая не давала открыть через обычную версию ChromeDriver (видела, что управляется через стороннее ПО и не давала открыть страницу). Данная версия ChromeDriver, пытается скрыть факт управляется браузером через Selenium. Это делает персер менее заметным для капчи Cloudflare

P.S.: Загрузка библиотек будет повторяться в коде, потому что мне так проще реализовывать части кода независимо друг от друга

In [88]:
import pandas as pd
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from datetime import datetime
import time

In [89]:
#Инициализация драйвера с использованием undetected_chromedriver
driver = uc.Chrome()

#URL темы для парсинга
url = "https://bitcointalk.org/index.php?topic=5509724.0;all"

driver.get(url)
time.sleep(10)  #Время на загрузку страницы и прохождение капчи

data = []  #Список для хранения данных

#Функция для парсинга страницы и сохранения комментариев
def parse_page():
    rows = driver.find_elements(By.XPATH, '//*[@id="quickModForm"]/table/tbody/tr')  # Поиск всех строк с комментариями
    for i, row in enumerate(rows, start=1):
        try:
            #Имя пользователя
            try:
                author_xpath = f'//*[@id="quickModForm"]/table/tbody/tr[{i}]/td/table/tbody/tr/td/table/tbody/tr[1]/td[1]/b/a'
                author = driver.find_element(By.XPATH, author_xpath).text.strip()
            except:
                author = None

            #Звание на форуме
            try:
                rank_xpath = f'//*[@id="quickModForm"]/table/tbody/tr[{i}]/td/table/tbody/tr/td/table/tbody/tr[1]/td[1]/div'
                rank_element = driver.find_element(By.XPATH, rank_xpath)
                rank = rank_element.text.split("\n")[0].strip()  # Извлечение текстового значения из элемента
            except:
                rank = None

            #Текст комментария и цитаты
            try:
                content_xpath = f'//*[@id="quickModForm"]/table/tbody/tr[{i}]/td/table/tbody/tr/td/table/tbody/tr[1]/td[2]/div'
                content_element = driver.find_element(By.XPATH, content_xpath)
                quote_headers = content_element.find_elements(By.CLASS_NAME, 'quoteheader')
                quotes = []
                for quote_header in quote_headers:
                    quote_text = quote_header.find_element(By.XPATH, './following-sibling::div[@class="quote"]').text.strip()
                    quotes.append(quote_header.text.strip() + "\n" + quote_text)
                quotes_text = "\n\n".join(quotes) if quotes else None
                content = content_element.text
                for quote_header in quote_headers:
                    content = content.replace(quote_header.text + "\n" + quote_header.find_element(By.XPATH, './following-sibling::div[@class="quote"]').text, "")  # Удаление текста цитаты из комментария
                content = content.strip()
            except:
                content = None
                quotes_text = None

            #Дата и время поста
            try:
                datetime_xpath = f'//*[@id="quickModForm"]/table/tbody/tr[{i}]/td/table/tbody/tr/td/table/tbody/tr[1]/td[2]/table/tbody/tr/td[2]/div[2]'
                datetime_text = driver.find_element(By.XPATH, datetime_xpath).text.strip()

                if "Today at" in datetime_text:
                    today = datetime.now()
                    time_str = datetime_text.replace("Today at ", "").strip()
                    datetime_obj = datetime.strptime(time_str, "%I:%M:%S %p")
                    post_date = today.strftime("%d.%m.%Y")
                    post_time = datetime_obj.strftime("%I:%M:%S %p")
                else:
                    datetime_obj = datetime.strptime(datetime_text, "%B %d, %Y, %I:%M:%S %p")
                    post_date = datetime_obj.strftime("%d.%m.%Y")
                    post_time = datetime_obj.strftime("%I:%M:%S %p")
            except:
                post_date = None
                post_time = None

            #Номер поста
            try:
                post_number_xpath = f'//*[@id="ignmsgbttns{i}"]/a'
                post_number = driver.find_element(By.XPATH, post_number_xpath).text.strip().replace('#', '')
            except:
                post_number = None

            #Сохранение данных, включая пустые цитаты
            data.append([post_number, post_date, post_time, author, rank, content, quotes_text])

        except Exception as e:
            print(f"Ошибка при парсинге строки {i}: {e}")

parse_page()  #Парсинг текущей страницы

driver.quit()  #Закрытие браузера

#Сохранение данных в df_example и Excel
if data:
    df_example = pd.DataFrame(data, columns=['post_number', 'post_date', 'post_time', 'author', 'rank', 'content', 'quotes'])
    #Удаление полностью пустых строк
    df_example.dropna(how='all', inplace=True)
    df_example.to_excel('bitcointalk_topic_comments.xlsx', index=False)
    print("Данные успешно сохранены в файл bitcointalk_topic_comments.xlsx")
else:
    print("Данные не были собраны или все записи оказались пустыми.")

Данные успешно сохранены в файл bitcointalk_topic_comments.xlsx


In [90]:
df_example

,post_number,post_date,post_time,author,rank,content,quotes
0,1,19.09.2024,07:03:41 PM,aima55,Jr. Member,Ton continues to attract investors as it is ra...,None
1,2,20.09.2024,11:57:28 PM,Tipstar,Sr. Member,Cardano had been an overhypped old and obsolet...,None
2,3,21.09.2024,12:22:03 AM,DeFi Better,Newbie,Telegram is amazing for several reasons. I per...,"Quote from: aima55 on September 19, 2024, 07:0..."
3,4,21.09.2024,02:38:41 AM,TastyChillySauce00,Legendary,Considering that binance keep on listing any T...,None
4,5,21.09.2024,07:17:05 AM,dansus021,Copper Member,First of all TON developer doenst really need ...,None
5,6,21.09.2024,10:01:04 AM,coin-investor,Hero Member,Ton was not on the radar of many investors but...,"Quote from: aima55 on September 19, 2024, 07:0..."
6,7,21.09.2024,10:08:13 AM,Iamcrypticguy,Jr. Member,I agree with your point that they need more qu...,"Quote from: DeFi Better on September 21, 2024,..."
7,8,22.09.2024,01:36:35 AM,TastyChillySauce00,Legendary,"TON blockchain still need cash, just recently ...","Quote from: dansus021 on September 21, 2024, 0..."
8,9,22.09.2024,11:26:10 AM,pooya87,Legendary,The only reason why TON network has gotten som...,None
9,10,23.09.2024,12:18:42 PM,dansus021,Copper Member,To be honest I still didn't understand with re...,Quote from: TastyChillySauce00 on September 22...


_С ссылки я скачиваю номер темы, номер поста, дату и время поста, автора, ранг автора на сайте, сам пост (на данном форуме любой комментарий в обсуждении является постом) и цитаты, если они есть. Важно, что бывают ошибки с отделением цитат от основного комментария (строка 37). Я думаю это проще вручную проверить позже \
**Нужно ли реализовывать дерево обсуждений или можно просто удалить цитаты и смотреть на окраску самого комментария без контекста?**_

### Загрузка комментариев

#### Скачивание ссылок на темы за последние 13 месяцев
Это примерно 64 страницы сайта

In [10]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import time

In [11]:
chrome_options = Options()
chrome_options.add_argument("--headless")  #Парсинг в фоновом режиме

driver = webdriver.Chrome(options=chrome_options)

#URL для парсинга
base_url = "https://bitcointalk.org/index.php?board=67.{}"
num_pages = 64

#Список для хранения данных
data = []

#Функция для парсинга страницы
def parse_page():
    rows = driver.find_elements(By.XPATH, '//*[@id="bodyarea"]/div[2]/table/tbody/tr')  # Поиск всех строк с темами
    for i, row in enumerate(rows, start=1):
        try:
            #Тема и ссылка
            try:
                title_element = driver.find_element(By.XPATH, f'//*[@id="bodyarea"]/div[2]/table/tbody/tr[{i}]/td[3]/span/a')
                title = title_element.text.strip()
                link = title_element.get_attribute("href")

                #Номер темы из ссылки
                topic_number = link.split("topic=")[1].split(".")[0]

            except NoSuchElementException:
                continue

            #Автор
            try:
                author_xpath = f'//*[@id="bodyarea"]/div[2]/table/tbody/tr[{i}]/td[4]/a'
                author = driver.find_element(By.XPATH, author_xpath).text.strip()
            except NoSuchElementException:
                continue

            #Добавление данных в список
            data.append([topic_number, title, author, link])

        except Exception as e:
            print(f"Error parsing row {i}: {e}")

#Перебор страниц
for page in range(0, num_pages):
    url = f"https://bitcointalk.org/index.php?board=67.{page * 40}"
    driver.get(url)
    time.sleep(2)  #Подождать, чтобы страница полностью загрузилась
    parse_page()

driver.quit()

#Сохранение данных в df_topics и Excel
df_topics = pd.DataFrame(data, columns=['topic_number', 'topic_title', 'author', 'link'])
df_topics.to_excel('bitcointalk_topics_13M.xlsx', index=False)

In [12]:
df_topics

,topic_number,topic_title,author,link
0,42465,Alternative Block Chains : be safe!,Gavin Andresen,https://bitcointalk.org/index.php?topic=42465.0
1,935898,Beware of Increasingly Sophisticated Malware I...,grue,https://bitcointalk.org/index.php?topic=935898.0
2,434310,Giveaway threads are not allowed,theymos,https://bitcointalk.org/index.php?topic=434310.0
3,1042322,Unofficial list of (official) Bitcointalk.org ...,mprep,https://bitcointalk.org/index.php?topic=1042322.0
4,5516460,Can Polkadot (DOT) coin can be the next Ethure...,Meuh5598i,https://bitcointalk.org/index.php?topic=5516460.0
...,...,...,...,...
2555,5467655,NounsDAO BAYC auctions have lost them over 100...,Vitalitik,https://bitcointalk.org/index.php?topic=5467655.0
2556,5467621,CSAD Blockchain,cybersecurityad,https://bitcointalk.org/index.php?topic=5467621.0
2557,5467618,MuskyDoge adding a Sportsbook,gmdisnuts,https://bitcointalk.org/index.php?topic=5467618.0
2558,5467460,Could SocialFi be the new trend in crypto?,coin-score.ai,https://bitcointalk.org/index.php?topic=5467460.0


#### Загрузка комментариев с обсуждений (всех) ПРИМЕР
Протестировано на небольшой выборке (5 тем)

In [41]:
import pandas as pd
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from datetime import datetime
import time

In [12]:
#df_topics = pd.read_excel('bitcointalk_topics_13M.xlsx')

In [42]:
# Загрузка данных из Excel (пример загрузки файла)
df_topics = pd.read_excel('bitcointalk_topics_mini.xlsx')

# Добавление ";all" к каждой ссылке в столбце 'link'
df_topics['link'] = df_topics['link'].apply(lambda x: x.replace('.0', '.0;all'))

In [49]:
df_topics

,topic_number,topic_title,author,link
0,5516460,Can Polkadot (DOT) coin can be the next Ethure...,Meuh5598i,https://bitcointalk.org/index.php?topic=551646...
1,5516023,Could $GRASS Be Leading the Next Wave of Decen...,3starcoin,https://bitcointalk.org/index.php?topic=551602...
2,5492669,Decentraland turning into SHITCOIN! SHITTIEST ...,CRAIC,https://bitcointalk.org/index.php?topic=549266...
3,5516468,What’s going on with Altcoins?,RaraAvis,https://bitcointalk.org/index.php?topic=551646...
4,5516588,Atomic wallet sometimes display absurd amounts...,AHOYBRAUSE,https://bitcointalk.org/index.php?topic=551658...


In [94]:
# Инициализация драйвера с использованием undetected_chromedriver
driver = uc.Chrome()

# Список для хранения данных
all_content_data = []

# Функция для парсинга страницы и сохранения комментариев
def parse_topic(url, topic_number):
    driver.get(url)
    time.sleep(5)  # Даем время на загрузку страницы
    data = []

    rows = driver.find_elements(By.XPATH, '//*[@id="quickModForm"]/table/tbody/tr')  # Поиск всех строк с комментариями
    for i, row in enumerate(rows, start=1):
        try:
            # Имя пользователя
            try:
                author_xpath = f'//*[@id="quickModForm"]/table/tbody/tr[{i}]/td/table/tbody/tr/td/table/tbody/tr[1]/td[1]/b/a'
                author = driver.find_element(By.XPATH, author_xpath).text.strip()
            except:
                author = None

            # Звание на форуме
            try:
                rank_xpath = f'//*[@id="quickModForm"]/table/tbody/tr[{i}]/td/table/tbody/tr/td/table/tbody/tr[1]/td[1]/div'
                rank_element = driver.find_element(By.XPATH, rank_xpath)
                rank = rank_element.text.split("\n")[0].strip()  # Извлечение текстового значения из элемента
            except:
                rank = None

            # Текст комментария и цитаты
            try:
                content_xpath = f'//*[@id="quickModForm"]/table/tbody/tr[{i}]/td/table/tbody/tr/td/table/tbody/tr[1]/td[2]/div'
                content_element = driver.find_element(By.XPATH, content_xpath)
                quote_headers = content_element.find_elements(By.CLASS_NAME, 'quoteheader')
                quotes = []
                for quote_header in quote_headers:
                    quote_text = quote_header.find_element(By.XPATH, './following-sibling::div[@class="quote"]').text.strip()
                    quotes.append(quote_header.text.strip() + "\n" + quote_text)
                quotes_text = "\n\n".join(quotes) if quotes else None
                content = content_element.text
                for quote_header in quote_headers:
                    content = content.replace(quote_header.text + "\n" + quote_header.find_element(By.XPATH, './following-sibling::div[@class="quote"]').text, "")  # Удаление текста цитаты из комментария
                content = content.strip()
            except:
                content = None
                quotes_text = None

            # Дата и время поста
            try:
                datetime_xpath = f'//*[@id="quickModForm"]/table/tbody/tr[{i}]/td/table/tbody/tr/td/table/tbody/tr[1]/td[2]/table/tbody/tr/td[2]/div[2]'
                datetime_text = driver.find_element(By.XPATH, datetime_xpath).text.strip()

                if "Today at" in datetime_text:
                    today = datetime.now()
                    time_str = datetime_text.replace("Today at ", "").strip()
                    datetime_obj = datetime.strptime(time_str, "%I:%M:%S %p")
                    post_date = today.strftime("%d.%m.%Y")
                    post_time = datetime_obj.strftime("%I:%M:%S %p")
                else:
                    datetime_obj = datetime.strptime(datetime_text, "%B %d, %Y, %I:%M:%S %p")
                    post_date = datetime_obj.strftime("%d.%m.%Y")
                    post_time = datetime_obj.strftime("%I:%M:%S %p")
            except:
                post_date = None
                post_time = None

            # Номер поста
            try:
                post_number_xpath = f'//*[@id="ignmsgbttns{i}"]/a'
                post_number = driver.find_element(By.XPATH, post_number_xpath).text.strip().replace('#', '')
            except:
                post_number = None

            # Сохранение данных, включая пустые цитаты
            data.append([topic_number, post_number, post_date, post_time, author, rank, content, quotes_text])

        except Exception as e:
            print(f"Ошибка при парсинге строки {i}: {e}")

    return data

# Перебор всех ссылок из df_topics и парсинг обсуждений
for index, row in df_topics.iterrows():
    link = row['link']
    topic_number = row['topic_number']
    content_data = parse_topic(link, topic_number)
    all_content_data.extend(content_data)

# Закрытие браузера
driver.quit()

# Сохранение всех данных в DataFrame и Excel
if all_content_data:
    df_content = pd.DataFrame(all_content_data, columns=['topic_number', 'post_number', 'post_date', 'post_time', 'author', 'rank', 'content', 'quotes'])
    # Удаление полностью пустых строк, кроме topic_number
    df_content = df_content.dropna(subset=['post_number', 'post_date', 'post_time', 'author', 'rank', 'content', 'quotes'], how='all')
    df_content.to_excel('bitcointalk_topics_content.xlsx', index=False)
    print("Данные успешно сохранены в файл bitcointalk_topics_content.xlsx")
else:
    print("Данные не были собраны или все записи оказались пустыми.")

Данные успешно сохранены в файл bitcointalk_topics_content.xlsx


In [95]:
df_content

,topic_number,post_number,post_date,post_time,author,rank,content,quotes
0,5516460,1,02.11.2024,06:06:27 AM,Meuh5598i,Full Member,"Hello Good Folk,\nI have been a little bit far...",None
1,5516460,2,02.11.2024,07:52:33 AM,hugeblack,Legendary,"NO, Polkadot (DOT) failed to be an alternative...",None
2,5516460,3,02.11.2024,08:40:31 AM,Sim_card,Hero Member,No coin can be an equivalent to bitcoin becaus...,"Quote from: Meuh5598i on November 02, 2024, 06..."
3,5516460,4,02.11.2024,09:52:36 AM,pakhitheboss,Hero Member,It was named as an Ethereum killer when it was...,None
4,5516460,5,02.11.2024,01:44:22 PM,MAAManda,Hero Member,Back when I first heard about Polkadot (DOT) &...,None
...,...,...,...,...,...,...,...,...
208,5516588,3,03.11.2024,08:40:31 PM,Charles-Tim,Legendary,It could be a hackers means to do something ju...,None
209,5516588,4,03.11.2024,08:54:19 PM,passwordnow,Hero Member,"A bug I guess, the same thing that some unknow...",None
210,5516588,5,04.11.2024,12:07:52 AM,X-ray,Hero Member,definitely just a mistake on the software part...,None
211,5516588,6,04.11.2024,10:23:53 AM,AHOYBRAUSE,Hero Member,Yeah obviously it is some faulty stuff with th...,Quote from: X-ray on Today at 12:07:52 AM\ndef...


#### Загрузка комментариев с обсуждений (упоминание TON/Telegram в самом обсуждении) ПРИМЕР
В данном случае использовался фильтр на "ton", "toncoin", "telegram" в тексте комментариев каждого обсуждения вне зависимости от регистра

In [36]:
import pandas as pd
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from datetime import datetime
import time

In [37]:
# Загрузка данных из Excel (пример загрузки файла)
df_topics = pd.read_excel('bitcointalk_topics_mini.xlsx')

# Добавление ";all" к каждой ссылке в столбце 'link'
df_topics['link'] = df_topics['link'].apply(lambda x: x.replace('.0', '.0;all'))

In [50]:
df_topics

,topic_number,topic_title,author,link
0,5516460,Can Polkadot (DOT) coin can be the next Ethure...,Meuh5598i,https://bitcointalk.org/index.php?topic=551646...
1,5516023,Could $GRASS Be Leading the Next Wave of Decen...,3starcoin,https://bitcointalk.org/index.php?topic=551602...
2,5492669,Decentraland turning into SHITCOIN! SHITTIEST ...,CRAIC,https://bitcointalk.org/index.php?topic=549266...
3,5516468,What’s going on with Altcoins?,RaraAvis,https://bitcointalk.org/index.php?topic=551646...
4,5516588,Atomic wallet sometimes display absurd amounts...,AHOYBRAUSE,https://bitcointalk.org/index.php?topic=551658...


In [103]:
# Инициализация драйвера с использованием undetected_chromedriver
driver = uc.Chrome()

# Список для хранения данных
all_content_data = []

# Функция для парсинга страницы и сохранения комментариев
def parse_topic(url, topic_number):
    driver.get(url)
    time.sleep(5)  # Даем время на загрузку страницы
    data = []

    rows = driver.find_elements(By.XPATH, '//*[@id="quickModForm"]/table/tbody/tr')  # Поиск всех строк с комментариями
    for i, row in enumerate(rows, start=1):
        try:
            # Имя пользователя
            try:
                author_xpath = f'//*[@id="quickModForm"]/table/tbody/tr[{i}]/td/table/tbody/tr/td/table/tbody/tr[1]/td[1]/b/a'
                author = driver.find_element(By.XPATH, author_xpath).text.strip()
            except:
                author = None

            # Звание на форуме
            try:
                rank_xpath = f'//*[@id="quickModForm"]/table/tbody/tr[{i}]/td/table/tbody/tr/td/table/tbody/tr[1]/td[1]/div'
                rank_element = driver.find_element(By.XPATH, rank_xpath)
                rank = rank_element.text.split("\n")[0].strip()  # Извлечение текстового значения из элемента
            except:
                rank = None

            # Текст комментария и цитаты
            try:
                content_xpath = f'//*[@id="quickModForm"]/table/tbody/tr[{i}]/td/table/tbody/tr/td/table/tbody/tr[1]/td[2]/div'
                content_element = driver.find_element(By.XPATH, content_xpath)
                quote_headers = content_element.find_elements(By.CLASS_NAME, 'quoteheader')
                quotes = []
                for quote_header in quote_headers:
                    quote_text = quote_header.find_element(By.XPATH, './following-sibling::div[@class="quote"]').text.strip()
                    quotes.append(quote_header.text.strip() + "\n" + quote_text)
                quotes_text = "\n\n".join(quotes) if quotes else None
                content = content_element.text
                for quote_header in quote_headers:
                    content = content.replace(quote_header.text + "\n" + quote_header.find_element(By.XPATH, './following-sibling::div[@class="quote"]').text, "")  # Удаление текста цитаты из комментария
                content = content.strip()
            except:
                content = None
                quotes_text = None

            # Дата и время поста
            try:
                datetime_xpath = f'//*[@id="quickModForm"]/table/tbody/tr[{i}]/td/table/tbody/tr/td/table/tbody/tr[1]/td[2]/table/tbody/tr/td[2]/div[2]'
                datetime_text = driver.find_element(By.XPATH, datetime_xpath).text.strip()

                if "Today at" in datetime_text:
                    today = datetime.now()
                    time_str = datetime_text.replace("Today at ", "").strip()
                    datetime_obj = datetime.strptime(time_str, "%I:%M:%S %p")
                    post_date = today.strftime("%d.%m.%Y")
                    post_time = datetime_obj.strftime("%I:%M:%S %p")
                else:
                    datetime_obj = datetime.strptime(datetime_text, "%B %d, %Y, %I:%M:%S %p")
                    post_date = datetime_obj.strftime("%d.%m.%Y")
                    post_time = datetime_obj.strftime("%I:%M:%S %p")
            except:
                post_date = None
                post_time = None

            # Номер поста
            try:
                post_number_xpath = f'//*[@id="ignmsgbttns{i}"]/a'
                post_number = driver.find_element(By.XPATH, post_number_xpath).text.strip().replace('#', '')
            except:
                post_number = None

            # Сохранение данных, включая пустые цитаты
            data.append([topic_number, post_number, post_date, post_time, author, rank, content, quotes_text])

        except Exception as e:
            print(f"Ошибка при парсинге строки {i}: {e}")

    return data

# Перебор всех ссылок из df_topics и парсинг обсуждений
for index, row in df_topics.iterrows():
    link = row['link']
    topic_number = row['topic_number']
    content_data = parse_topic(link, topic_number)
    all_content_data.extend(content_data)

# Закрытие браузера
driver.quit()

# Сохранение всех данных в DataFrame и Excel
if all_content_data:
    df_content_all = pd.DataFrame(all_content_data, columns=['topic_number', 'post_number', 'post_date', 'post_time', 'author', 'rank', 'content', 'quotes'])
    #Удаление полностью пустых строк, кроме topic_number
    df_content_all = df_content_all.dropna(subset=['post_number', 'post_date', 'post_time', 'author', 'rank', 'content', 'quotes'], how='all')
    df_content_all.to_excel('bitcointalk_topics_content_all.xlsx', index=False)
    print("Данные успешно сохранены в файл bitcointalk_topics_content_all.xlsx")
else:
    print("Данные не были собраны или все записи оказались пустыми.")

Данные успешно сохранены в файл bitcointalk_topics_content.xlsx


In [108]:
df_content_all

,topic_number,post_number,post_date,post_time,author,rank,content,quotes
0,5516460,1,02.11.2024,06:06:27 AM,Meuh5598i,Full Member,"Hello Good Folk,\nI have been a little bit far...",NaN
1,5516460,2,02.11.2024,07:52:33 AM,hugeblack,Legendary,"NO, Polkadot (DOT) failed to be an alternative...",NaN
2,5516460,3,02.11.2024,08:40:31 AM,Sim_card,Hero Member,No coin can be an equivalent to bitcoin becaus...,"Quote from: Meuh5598i on November 02, 2024, 06..."
3,5516460,4,02.11.2024,09:52:36 AM,pakhitheboss,Hero Member,It was named as an Ethereum killer when it was...,NaN
4,5516460,5,02.11.2024,01:44:22 PM,MAAManda,Hero Member,Back when I first heard about Polkadot (DOT) &...,NaN
...,...,...,...,...,...,...,...,...
193,5516588,3,03.11.2024,08:40:31 PM,Charles-Tim,Legendary,It could be a hackers means to do something ju...,NaN
194,5516588,4,03.11.2024,08:54:19 PM,passwordnow,Hero Member,"A bug I guess, the same thing that some unknow...",NaN
195,5516588,5,04.11.2024,12:07:52 AM,X-ray,Hero Member,definitely just a mistake on the software part...,NaN
196,5516588,6,04.11.2024,10:23:53 AM,AHOYBRAUSE,Hero Member,Yeah obviously it is some faulty stuff with th...,Quote from: X-ray on Today at 12:07:52 AM\ndef...


In [104]:
df_content_all = pd.read_excel('bitcointalk_topics_content_all.xlsx')

In [106]:
#Создание временного DataFrame с темами в нижнем регистре только для столбца 'content'
df_content_temp = df_content_all.copy()
df_content_temp['content'] = df_content_temp['content'].str.lower().fillna('')

keywords = ["ton", "toncoin", "telegram"]  #Ключевые слова для фильтрации

#Фильтрация по ключевым словам в колонке 'content' временного DataFrame
filtered_df_content = df_content_all[df_content_temp['content'].str.contains('|'.join(keywords), na=False)]

#Сохранение отфильтрованных данных в Excel
filtered_df_content.to_excel('bitcointalk_topics_content_all_filtered.xlsx', index=False)

In [107]:
filtered_df_content

,topic_number,post_number,post_date,post_time,author,rank,content,quotes
39,5516023,18,04.11.2024,03:56:25 PM,pakhitheboss,Hero Member,"I participated in their program, but I feel I ...",NaN
149,5492669,110,22.10.2024,04:51:56 AM,Decentraland (MANA),Newbie,entrypoint is today\n\nDecentraland 2.0 to Rel...,"Quote from: markm on September 20, 2024, 12:22..."
189,5516468,26,04.11.2024,05:39:56 PM,Z_MBFM,Sr. Member,Bitcoin price is moving abnormally because of ...,"Quote from: RaraAvis on November 02, 2024, 10:..."
191,5516588,1,03.11.2024,07:09:35 PM,AHOYBRAUSE,Hero Member,So I just logged into my atomic wallet on desk...,NaN
192,5516588,2,03.11.2024,07:39:59 PM,Stalker22,Legendary,"Quote from: AHOYBRAUSE on November 03, 2024, 0...","Quote from: AHOYBRAUSE on November 03, 2024, 0..."
193,5516588,3,03.11.2024,08:40:31 PM,Charles-Tim,Legendary,It could be a hackers means to do something ju...,NaN
197,5516588,7,04.11.2024,10:40:45 AM,GxSTxV,Hero Member,I Personally have seen similar case with MetaM...,NaN


_Было бы красивее реализовать поиск с сайта и сразу добавление в df только комментариев с TON, но мне проще на данный момент скачать все и дальше сделать фильтр. В коде спользуется фильтр на "ton", "toncoin", "telegram" в тексте комментариев каждого обсуждения вне зависимости от регистра_

#### Загрузка комментариев с обсуждений (упоминание TON/Telegram в самом обсуждении) ПОЛНОСТЬЮ

In [109]:
import pandas as pd
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from datetime import datetime
import time

In [110]:
# Загрузка данных из Excel (пример загрузки файла)
df_topics = pd.read_excel('bitcointalk_topics_13M.xlsx')

# Добавление ";all" к каждой ссылке в столбце 'link'
df_topics['link'] = df_topics['link'].apply(lambda x: x.replace('.0', '.0;all'))

In [111]:
df_topics

,topic_number,topic_title,author,link
0,42465,Alternative Block Chains : be safe!,Gavin Andresen,https://bitcointalk.org/index.php?topic=42465....
1,935898,Beware of Increasingly Sophisticated Malware I...,grue,https://bitcointalk.org/index.php?topic=935898...
2,434310,Giveaway threads are not allowed,theymos,https://bitcointalk.org/index.php?topic=434310...
3,1042322,Unofficial list of (official) Bitcointalk.org ...,mprep,https://bitcointalk.org/index.php?topic=104232...
4,5516460,Can Polkadot (DOT) coin can be the next Ethure...,Meuh5598i,https://bitcointalk.org/index.php?topic=551646...
...,...,...,...,...
2555,5467655,NounsDAO BAYC auctions have lost them over 100...,Vitalitik,https://bitcointalk.org/index.php?topic=546765...
2556,5467621,CSAD Blockchain,cybersecurityad,https://bitcointalk.org/index.php?topic=546762...
2557,5467618,MuskyDoge adding a Sportsbook,gmdisnuts,https://bitcointalk.org/index.php?topic=546761...
2558,5467460,Could SocialFi be the new trend in crypto?,coin-score.ai,https://bitcointalk.org/index.php?topic=546746...


In [113]:
# Инициализация драйвера с использованием undetected_chromedriver
driver = uc.Chrome()

# Список для хранения данных
all_content_data = []

# Функция для парсинга страницы и сохранения комментариев
def parse_topic(url, topic_number):
    driver.get(url)
    time.sleep(5)  # Даем время на загрузку страницы
    data = []

    rows = driver.find_elements(By.XPATH, '//*[@id="quickModForm"]/table/tbody/tr')  # Поиск всех строк с комментариями
    for i, row in enumerate(rows, start=1):
        try:
            # Имя пользователя
            try:
                author_xpath = f'//*[@id="quickModForm"]/table/tbody/tr[{i}]/td/table/tbody/tr/td/table/tbody/tr[1]/td[1]/b/a'
                author = driver.find_element(By.XPATH, author_xpath).text.strip()
            except:
                author = None

            # Звание на форуме
            try:
                rank_xpath = f'//*[@id="quickModForm"]/table/tbody/tr[{i}]/td/table/tbody/tr/td/table/tbody/tr[1]/td[1]/div'
                rank_element = driver.find_element(By.XPATH, rank_xpath)
                rank = rank_element.text.split("\n")[0].strip()  # Извлечение текстового значения из элемента
            except:
                rank = None

            # Текст комментария и цитаты
            try:
                content_xpath = f'//*[@id="quickModForm"]/table/tbody/tr[{i}]/td/table/tbody/tr/td/table/tbody/tr[1]/td[2]/div'
                content_element = driver.find_element(By.XPATH, content_xpath)
                quote_headers = content_element.find_elements(By.CLASS_NAME, 'quoteheader')
                quotes = []
                for quote_header in quote_headers:
                    quote_text = quote_header.find_element(By.XPATH, './following-sibling::div[@class="quote"]').text.strip()
                    quotes.append(quote_header.text.strip() + "\n" + quote_text)
                quotes_text = "\n\n".join(quotes) if quotes else None
                content = content_element.text
                for quote_header in quote_headers:
                    content = content.replace(quote_header.text + "\n" + quote_header.find_element(By.XPATH, './following-sibling::div[@class="quote"]').text, "")  # Удаление текста цитаты из комментария
                content = content.strip()
            except:
                content = None
                quotes_text = None

            # Дата и время поста
            try:
                datetime_xpath = f'//*[@id="quickModForm"]/table/tbody/tr[{i}]/td/table/tbody/tr/td/table/tbody/tr[1]/td[2]/table/tbody/tr/td[2]/div[2]'
                datetime_text = driver.find_element(By.XPATH, datetime_xpath).text.strip()

                if "Today at" in datetime_text:
                    today = datetime.now()
                    time_str = datetime_text.replace("Today at ", "").strip()
                    datetime_obj = datetime.strptime(time_str, "%I:%M:%S %p")
                    post_date = today.strftime("%d.%m.%Y")
                    post_time = datetime_obj.strftime("%I:%M:%S %p")
                else:
                    datetime_obj = datetime.strptime(datetime_text, "%B %d, %Y, %I:%M:%S %p")
                    post_date = datetime_obj.strftime("%d.%m.%Y")
                    post_time = datetime_obj.strftime("%I:%M:%S %p")
            except:
                post_date = None
                post_time = None

            # Номер поста
            try:
                post_number_xpath = f'//*[@id="ignmsgbttns{i}"]/a'
                post_number = driver.find_element(By.XPATH, post_number_xpath).text.strip().replace('#', '')
            except:
                post_number = None

            # Сохранение данных, включая пустые цитаты
            data.append([topic_number, post_number, post_date, post_time, author, rank, content, quotes_text])

        except Exception as e:
            print(f"Ошибка при парсинге строки {i}: {e}")

    return data

# Перебор всех ссылок из df_topics и парсинг обсуждений
for index, row in df_topics.iterrows():
    link = row['link']
    topic_number = row['topic_number']
    content_data = parse_topic(link, topic_number)
    all_content_data.extend(content_data)

# Закрытие браузера
driver.quit()

# Сохранение всех данных в DataFrame и Excel
if all_content_data:
    df_content_13M = pd.DataFrame(all_content_data, columns=['topic_number', 'post_number', 'post_date', 'post_time', 'author', 'rank', 'content', 'quotes'])
    #Удаление полностью пустых строк, кроме topic_number
    df_content_13M = df_content_13M.dropna(subset=['post_number', 'post_date', 'post_time', 'author', 'rank', 'content', 'quotes'], how='all')
    df_content_13M.to_excel('bitcointalk_topics_content_13M.xlsx', index=False)
    print("Данные успешно сохранены в файл bitcointalk_topics_content_13M.xlsx")
else:
    print("Данные не были собраны или все записи оказались пустыми.")

IllegalCharacterError: 

_**Очень долгий код**_

In [114]:
df_content_13M

,topic_number,post_number,post_date,post_time,author,rank,content,quotes
0,42465,1,09.09.2011,01:21:18 PM,Gavin Andresen,Legendary,I haven't seen anybody post about what would b...,None
1,42465,2,09.09.2011,01:48:36 PM,Lolcust,Member,Thank you Gavin.\n\nThe only things I might ad...,None
2,42465,3,09.09.2011,02:08:16 PM,makomk,Hero Member,"Also, don't believe everything that prominent ...",None
3,42465,4,09.09.2011,02:32:36 PM,bitlotto,Hero Member,Good advice.\n\nUsing an alternate cryptocurre...,None
4,42465,5,09.09.2011,05:04:08 PM,Lolcust,Member,"Generally, if you have a large amount of bitco...",None
...,...,...,...,...,...,...,...,...
90509,5467460,10,21.09.2023,08:55:45 AM,blockman,Hero Member,There's a lack of interest from the community ...,"Quote from: Yudhisthir on September 20, 2023, ..."
90510,5467460,11,21.09.2023,09:13:02 AM,Text,Hero Member,I believe that SocialFi will be the new trend ...,None
90511,5467460,12,21.09.2023,03:23:20 PM,LogitechMouse,Legendary,SocialFi? New trend in crypto? What is that?\n...,"Quote from: coin-score.ai on September 20, 202..."
90516,5467574,1,21.09.2023,11:48:09 AM,xDeFixSolutionSx,Copper Member,"Sony has partnered with Startale Labs, a Singa...",None


_Код выполнялся очень долго. Порядка 9 часов. \
Данные сохранились в **df_content_13M**, но при сохранении в Excel вышла ошибка. В связи с чем я сохранила их вручную через clipboard и назвала файл **'bitcointalk_topics_content_13M_full.xlsx'**_

In [116]:
df_content_13M.to_clipboard(excel=True)

In [146]:
len(df_content_13M)

80282

_Мне необходимо очистить весь этот набор данных. Нужно ввести фильтры_

**Фильтр по ключевым словам "ton", "toncoin", "telegram" \
Фильтр по датам с 20.09.2023 до 04.11.2024**

In [ ]:
#df_content_13M = pd.read_excel('bitcointalk_topics_content_13M_full.xlsx')

In [137]:
import pandas as pd
import re

In [142]:
#Создание временного DataFrame с темами в нижнем регистре только для столбца 'content'
df_content_temp = df_content_13M.copy()
df_content_temp['content'] = df_content_temp['content'].str.lower().fillna('')

keywords = ["ton", "toncoin", "telegram"]  #Ключевые слова для фильтрации

#Создание регулярного выражения для поиска полных слов
keywords_pattern = r'\b(' + '|'.join(re.escape(keyword) for keyword in keywords) + r')\b'

#Фильтрация по ключевым словам в колонке 'content' временного DataFrame
filtered_df_content = df_content_13M[df_content_temp['content'].str.contains(keywords_pattern, na=False, regex=True)]

#Преобразование столбца 'post_date' в datetime
filtered_df_content['post_date'] = pd.to_datetime(filtered_df_content['post_date'], format='%d.%m.%Y', errors='coerce')

#Удаление строк, где дата не соответствует диапазону от 20.09.2023 до 04.11.2024
filtered_df_content = filtered_df_content[(filtered_df_content['post_date'] >= '2023-09-20') & (filtered_df_content['post_date'] <= '2024-11-04')]

#Сохранение отфильтрованных данных в Excel
filtered_df_content.to_excel('bitcointalk_topics_content_13M_full_filtered.xlsx', index=False)

In [143]:
filtered_df_content

,topic_number,post_number,post_date,post_time,author,rank,content,quotes
120,5516023,18,2024-11-04,03:56:25 PM,pakhitheboss,Hero Member,"I participated in their program, but I feel I ...",None
278,5516468,26,2024-11-04,05:39:56 PM,Z_MBFM,Sr. Member,Bitcoin price is moving abnormally because of ...,"Quote from: RaraAvis on November 02, 2024, 10:..."
307,5513355,6,2024-10-16,06:26:26 AM,Barikui1,Sr. Member,"Quote from: aima55 on October 15, 2024, 11:12:...","Quote from: aima55 on October 15, 2024, 11:12:..."
320,5513355,19,2024-10-16,05:39:01 PM,danherbias07,Legendary,You are frustrated because you are expecting s...,None
324,5513355,23,2024-10-16,09:45:12 PM,Furious 7,Hero Member,Talking about airdrops that are currently cons...,None
...,...,...,...,...,...,...,...,...
90461,5466774,52,2023-09-22,04:18:20 AM,RussianEnglishTranslation,Jr. Member,"Quote from: CryptSafe on September 14, 2023, 1...","Quote from: CryptSafe on September 14, 2023, 1..."
90462,5466774,53,2023-09-22,07:15:27 AM,X-ray,Hero Member,How safe telegram wallet is still questionable...,"Quote from: Husseysamuel on September 21, 2023..."
90463,5466774,54,2023-09-22,08:29:06 AM,gurunanakji777,Full Member,"I would say it's good news for Telegram users,...",None
90465,5466774,56,2023-09-22,11:04:14 AM,CryptSafe,Hero Member,Telegram security features needs to be upgrade...,"Quote from: gurunanakji777 on September 22, 20..."


In [140]:
len(filtered_df_content)

2974

_Итоговый **filtered_df_content** включает 3526 комментариев. Однако это включая "telegram". Нужен ли фильтр по данному слову?_

In [144]:
#Создание временного DataFrame с темами в нижнем регистре только для столбца 'content'
df_content_temp = df_content_13M.copy()
df_content_temp['content'] = df_content_temp['content'].str.lower().fillna('')

keywords = ["ton", "toncoin"]  #Ключевые слова для фильтрации

#Создание регулярного выражения для поиска полных слов
keywords_pattern = r'\b(' + '|'.join(re.escape(keyword) for keyword in keywords) + r')\b'

#Фильтрация по ключевым словам в колонке 'content' временного DataFrame
filtered_df_content_ton = df_content_13M[df_content_temp['content'].str.contains(keywords_pattern, na=False, regex=True)]

#Преобразование столбца 'post_date' в datetime
filtered_df_content_ton['post_date'] = pd.to_datetime(filtered_df_content_ton['post_date'], format='%d.%m.%Y', errors='coerce')

#Удаление строк, где дата не соответствует диапазону от 20.09.2023 до 04.11.2024
filtered_df_content_ton = filtered_df_content_ton[(filtered_df_content_ton['post_date'] >= '2023-09-20') & (filtered_df_content_ton['post_date'] <= '2024-11-04')]

#Сохранение отфильтрованных данных в Excel
filtered_df_content_ton.to_excel('bitcointalk_topics_content_13M_full_filtered_ton.xlsx', index=False)

In [145]:
len(filtered_df_content_ton)

1523

_Без упоминания "telegram" **filtered_df_content_ton** включает всего 1523 комментария_

### Ошибки с цитатами

Какое-то количество раз в столбец 'content' попадали цитаты. Они сохраняются как в 'quotes', так и в 'content'. Необходимо удалить повторы из 'content'

In [2]:
import pandas as pd
import re

In [3]:
#Загрузка данных из Excel
filtered_df_content_ton = pd.read_excel('bitcointalk_topics_content_13M_full_filtered_ton.xlsx')

In [4]:
filtered_df_content_ton

,topic_number,post_number,post_date,post_time,author,rank,content,quotes
0,5516468,26,2024-11-04,05:39:56 PM,Z_MBFM,Sr. Member,Bitcoin price is moving abnormally because of ...,"Quote from: RaraAvis on November 02, 2024, 10:..."
1,5516588,1,2024-11-03,07:09:35 PM,AHOYBRAUSE,Hero Member,So I just logged into my atomic wallet on desk...,NaN
2,5516588,2,2024-11-03,07:39:59 PM,Stalker22,Legendary,"Quote from: AHOYBRAUSE on November 03, 2024, 0...","Quote from: AHOYBRAUSE on November 03, 2024, 0..."
3,5516588,3,2024-11-03,08:40:31 PM,Charles-Tim,Legendary,It could be a hackers means to do something ju...,NaN
4,5516588,7,2024-11-04,10:40:45 AM,GxSTxV,Hero Member,I Personally have seen similar case with MetaM...,NaN
...,...,...,...,...,...,...,...,...
1518,5466111,99,2023-09-21,10:58:09 PM,serjent05,Legendary,"Quote from: cabron on September 17, 2023, 07:5...","Quote from: cabron on September 17, 2023, 07:5..."
1519,5466111,101,2023-09-22,04:15:41 AM,RussianEnglishTranslation,Jr. Member,TON is actually going up right now. Check the ...,"Quote from: serjent05 on September 21, 2023, 1..."
1520,5466111,138,2023-10-02,05:55:53 PM,justdimin,Hero Member,"Understanding it is not the point, we should b...","Quote from: _Hiloveua_ on October 02, 2023, 12..."
1521,5468037,5,2023-09-29,01:37:43 AM,dansus021,Copper Member,It is a different case but yes whale can be de...,"Quote from: woobid on September 25, 2023, 01:1..."


In [5]:
#Создание копии DataFrame для работы
filtered_df_content_cleaned = filtered_df_content_ton.copy()

#Удаление пустых строк или строк, содержащих только пробелы/абзацы, из столбца 'content'
filtered_df_content_cleaned['content'] = filtered_df_content_cleaned['content'].apply(lambda x: '\n'.join([line for line in x.split('\n') 
                                                                                                         if line.strip() != '']) if pd.notna(x) else x)
#Удаление пустых строк или строк, содержащих только пробелы/абзацы, из столбца 'quotes'
filtered_df_content_cleaned['quotes'] = filtered_df_content_cleaned['quotes'].apply(lambda x: '\n'.join([line for line in x.split('\n') 
                                                                                                         if line.strip() != '']) if pd.notna(x) else x)

In [6]:
#Функция для удаления вложенных цитат и их точных совпадений
def remove_quotes_from_content(quote, content):
    #Удаляем лишние пробелы и символы новой строки из цитаты и контента перед сравнением
    quote_cleaned = re.sub(r'\s+', ' ', quote).strip()
    content_cleaned = re.sub(r'\s+', ' ', content).strip()

    #Если очищенная цитата присутствует в очищенном контенте, удаляем её
    while quote_cleaned in content_cleaned:
        content_cleaned = re.sub(re.escape(quote_cleaned), '', content_cleaned, count=1).strip()

    #Возвращаем очищенный контент
    return content_cleaned

#Удаление точных совпадений цитат из контента
for index, row in filtered_df_content_cleaned.iterrows():
    quote = row['quotes']
    content = row['content']

    if pd.notna(quote) and pd.notna(content):
        #Разделение всех цитат по ключевому слову "Quote from:"
        quote_parts = quote.split('Quote from:')
        for part in quote_parts:
            part = part.strip()
            if part:
                #Добавляем обратно "Quote from:" к каждой части и очищаем контент
                full_quote = 'Quote from: ' + part
                content = remove_quotes_from_content(full_quote, content)

        #Обновляем строку контента в DataFrame
        filtered_df_content_cleaned.at[index, 'content'] = content

#Сохранение обновленного df в Excel
filtered_df_content_cleaned.to_excel('bitcointalk_13M_ton.xlsx', index=False)

In [7]:
filtered_df_content_cleaned

,topic_number,post_number,post_date,post_time,author,rank,content,quotes
0,5516468,26,2024-11-04,05:39:56 PM,Z_MBFM,Sr. Member,Bitcoin price is moving abnormally because of ...,"Quote from: RaraAvis on November 02, 2024, 10:..."
1,5516588,1,2024-11-03,07:09:35 PM,AHOYBRAUSE,Hero Member,So I just logged into my atomic wallet on desk...,NaN
2,5516588,2,2024-11-03,07:39:59 PM,Stalker22,Legendary,How do you still feel comfortable using Atomic...,"Quote from: AHOYBRAUSE on November 03, 2024, 0..."
3,5516588,3,2024-11-03,08:40:31 PM,Charles-Tim,Legendary,It could be a hackers means to do something ju...,NaN
4,5516588,7,2024-11-04,10:40:45 AM,GxSTxV,Hero Member,I Personally have seen similar case with MetaM...,NaN
...,...,...,...,...,...,...,...,...
1518,5466111,99,2023-09-21,10:58:09 PM,serjent05,Legendary,"True that, there are other means of getting pr...","Quote from: cabron on September 17, 2023, 07:5..."
1519,5466111,101,2023-09-22,04:15:41 AM,RussianEnglishTranslation,Jr. Member,TON is actually going up right now. Check the ...,"Quote from: serjent05 on September 21, 2023, 1..."
1520,5466111,138,2023-10-02,05:55:53 PM,justdimin,Hero Member,"Understanding it is not the point, we should b...","Quote from: _Hiloveua_ on October 02, 2023, 12..."
1521,5468037,5,2023-09-29,01:37:43 AM,dansus021,Copper Member,It is a different case but yes whale can be de...,"Quote from: woobid on September 25, 2023, 01:1..."


Подсчет количества строк, в которых встречается слово 'quote' в столбце 'content'

In [8]:
#Количество строк в filtered_df_content_ton, содержащих слово 'quote'
count_ton = filtered_df_content_ton[filtered_df_content_ton['content'].str.contains('quote', case=False, na=False)].shape[0]
print(f"Количество строк в filtered_df_content_ton с 'quote' в столбце content: {count_ton}")

#Количество строк в filtered_df_content_cleaned, содержащих слово 'quote'
count_cleaned = filtered_df_content_cleaned[filtered_df_content_cleaned['content'].str.contains('quote', case=False, na=False)].shape[0]
print(f"Количество строк в filtered_df_content_cleaned с 'quote' в столбце content: {count_cleaned}")

Количество строк в filtered_df_content_ton с 'quote' в столбце content: 104
Количество строк в filtered_df_content_cleaned с 'quote' в столбце content: 3


In [11]:
print(len(filtered_df_content_ton))
print(len(filtered_df_content_cleaned))

1523
1523


In [16]:
#Создание временного DataFrame с темами в нижнем регистре только для столбца 'content'
filtered_df_content_temp = filtered_df_content_cleaned.copy()
filtered_df_content_temp['content'] = filtered_df_content_temp['content'].str.lower().fillna('')

keywords = ["ton", "toncoin", "telegram"]  #Ключевые слова для фильтрации

#Создание регулярного выражения для поиска полных слов
keywords_pattern = r'\b(' + '|'.join(re.escape(keyword) for keyword in keywords) + r')\b'

#Фильтрация по ключевым словам в колонке 'content' временного DataFrame
df_ton = filtered_df_content_cleaned[filtered_df_content_temp['content'].str.contains(keywords_pattern, na=False, regex=True)]

#Сохранение отфильтрованных данных в Excel
df_ton.to_excel('bitcointalk_13M_ton_final.xlsx', index=False)

In [17]:
print(len(df_ton))

1505


### ДОП

#### Загрузка комментариев с обсуждений (упоминание TON/Telegram в названии) ДОП

In [32]:
import pandas as pd

In [22]:
#Загрузка ранее собранного DataFrame с темами
df_topics = pd.read_excel('bitcointalk_topics_13M.xlsx')

In [23]:
#Создание временного DataFrame с темами в нижнем регистре
df_topics_temp = df_topics.copy()
df_topics_temp['topic_title'] = df_topics_temp['topic_title'].str.lower()
keywords = ["ton", "toncoin", "telegram"] #Ключевые слова для фильтрации

#Фильтрация тем, содержащих указанные ключевые слова
df_topics_ton = df_topics[df_topics_temp['topic_title'].str.contains('|'.join(keywords), na=False)]
#df_topics_ton.to_excel('bitcointalk_topics_ton.xlsx', index=False) #Сохранение отфильтрованного DataFrame в новый Excel-файл

In [24]:
df_topics_ton

,topic_number,topic_title,author,link
13,5510558,The Future of Telegram Airdrops After Hamster ...,Yucky,https://bitcointalk.org/index.php?topic=5510558.0
19,5509724,TON’s User Boom: Outclassing Ghost Chains Like...,aima55,https://bitcointalk.org/index.php?topic=5509724.0
46,5504749,Is Telegram The New Haven for Bounty Hunters?,ContentWriter,https://bitcointalk.org/index.php?topic=5504749.0
51,5511210,Telegram airdrops with potential,sale1143,https://bitcointalk.org/index.php?topic=5511210.0
65,5510767,Evaluating Recent Airdrops from Telegram Minin...,LiquidationBro,https://bitcointalk.org/index.php?topic=5510767.0
...,...,...,...,...
2198,5471980,Compiling CryptoNote Wallet for Windows,Psylocybin42,https://bitcointalk.org/index.php?topic=5471980.0
2310,5470069,NFT sale has already started #Nortonium,Nortonium,https://bitcointalk.org/index.php?topic=5470069.0
2419,5466774,Telegram launches crypto currency wallet.,CryptSafe,https://bitcointalk.org/index.php?topic=5466774.0
2459,5440392,The future of Toncoin,hack3rcon,https://bitcointalk.org/index.php?topic=5440392.0


In [33]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import time
from datetime import datetime

In [25]:
chrome_options = Options()
chrome_options.add_argument("--headless")  #Парсинг в фоновом режиме

driver = webdriver.Chrome(options=chrome_options)

#Функция для парсинга одной темы и получения обсуждения
def parse_topic(url):
    driver.get(url)
    time.sleep(2)  #Подождать, чтобы страница полностью загрузилась
    topic_number = url.split('=')[1].split('.')[0]
    
    data = []
    post_counter = 1  #Начальный счетчик для номеров сообщений

    #Функция для парсинга страницы обсуждения
    def parse_page():
        nonlocal post_counter
        rows = driver.find_elements(By.XPATH, '//*[@id="quickModForm"]/table/tbody/tr')  # Поиск всех строк с комментариями
        for i, row in enumerate(rows, start=1):
            try:
                #Имя пользователя
                try:
                    author_xpath = f'//*[@id="quickModForm"]/table/tbody/tr[{i}]/td/table/tbody/tr/td/table/tbody/tr[1]/td[1]/b/a'
                    author = driver.find_element(By.XPATH, author_xpath).text.strip()
                except NoSuchElementException:
                    continue  #Пропустить запись, если нет автора

                #Звание на форуме
                try:
                    rank_xpath = f'//*[@id="quickModForm"]/table/tbody/tr[{i}]/td/table/tbody/tr/td/table/tbody/tr[1]/td[1]/div'
                    rank_element = driver.find_element(By.XPATH, rank_xpath)
                    rank = rank_element.text.split("\n")[0].strip()  # Извлечение текстового значения из элемента
                except NoSuchElementException:
                    continue  #Пропустить запись, если нет звания

                #Текст комментария
                try:
                    content_element = driver.find_element(By.XPATH, f'//*[@id="quickModForm"]/table/tbody/tr[{i}]/td/table/tbody/tr/td/table/tbody/tr[1]/td[2]/div')
                    paragraphs = content_element.text.split("\n")
                    content = "\n".join(paragraphs)
                except NoSuchElementException:
                    continue  #Пропустить запись, если нет контента

                #Дата и время поста
                try:
                    datetime_xpath = f'//*[@id="quickModForm"]/table/tbody/tr[{i}]/td/table/tbody/tr/td/table/tbody/tr[1]/td[2]/table/tbody/tr/td[2]/div[2]'
                    datetime_text = driver.find_element(By.XPATH, datetime_xpath).text.strip()

                    if "Today at" in datetime_text:
                        today = datetime.now()
                        time_str = datetime_text.replace("Today at ", "").strip()
                        datetime_obj = datetime.strptime(time_str, "%I:%M:%S %p")
                        post_date = today.strftime("%d.%m.%Y")
                        post_time = datetime_obj.strftime("%I:%M:%S %p")
                    else:
                        datetime_obj = datetime.strptime(datetime_text, "%B %d, %Y, %I:%M:%S %p")
                        post_date = datetime_obj.strftime("%d.%m.%Y")
                        post_time = datetime_obj.strftime("%I:%M:%S %p")

                except NoSuchElementException:
                    continue  #Пропустить запись, если нет даты и времени

                #Сохранение данных, если поля не пустые
                data.append([topic_number, post_counter, post_date, post_time, author, rank, content])
                post_counter += 1  #Увеличение счетчика сообщений

            except Exception as e:
                print(f"Error parsing row {i}: {e}")

    #Парсинг первой страницы темы и всех её страниц (пока есть кнопка перехода)
    page_number = 1
    while True:
        parse_page()
        try:
            #Поиск ссылки на следующую страницу по её порядковому номеру
            next_page_xpath = f'//*[@id="bodyarea"]/table[1]/tbody/tr/td[1]/a[{page_number}]'
            next_page = driver.find_element(By.XPATH, next_page_xpath)
            next_page.click()
            page_number += 1
            time.sleep(2)  # Подождать, чтобы страница загрузилась
        except NoSuchElementException:
            #Если следующая страница не найдена, завершить цикл
            break

    return data

#Перебор всех ссылок из df_topics_ton и парсинг обсуждений
all_content_data = []
for index, row in df_topics_ton.iterrows():
    link = row['link']
    content_data = parse_topic(link)
    all_content_data.extend(content_data)

driver.quit()

#Сохранение всех данных в df_content_ton и Excel
df_content_ton = pd.DataFrame(all_content_data, columns=['topic_number', 'post_number', 'post_date', 'post_time', 'author', 'rank', 'content'])
df_content_ton.to_excel('bitcointalk_topics_ton.xlsx', index=False)

In [26]:
df_content_ton

,topic_number,post_number,post_date,post_time,author,rank,content
0,5510558,1,26.09.2024,02:28:52 PM,Yucky,Full Member,"Today, the Hamster airdrop project listed, and..."
1,5510558,2,26.09.2024,02:42:57 PM,Charles-Tim,Legendary,"Quote from: Yucky on September 26, 2024, 02:28..."
2,5510558,3,26.09.2024,04:14:05 PM,Tmoonz,Sr. Member,"Quote from: Yucky on September 26, 2024, 02:28..."
3,5510558,4,26.09.2024,04:20:09 PM,Adbitco,Hero Member,"Quote from: Charles-Tim on September 26, 2024,..."
4,5510558,5,26.09.2024,04:51:36 PM,salad daging,Hero Member,"Quote from: Yucky on September 26, 2024, 02:28..."
...,...,...,...,...,...,...,...
1494,5440392,11,13.09.2023,01:27:18 PM,covfefe_,Full Member,"Quote from: hack3rcon on February 19, 2023, 08..."
1495,5440392,12,13.09.2023,01:38:56 PM,zasad@,Legendary,"Quote from: hack3rcon on February 19, 2023, 08..."
1496,5440392,13,13.09.2023,08:29:16 PM,Hildentine,Jr. Member,I think every successful project are depends i...
1497,5440392,14,13.09.2023,09:59:19 PM,jossiel,Hero Member,"Quote from: rhomelmabini on September 13, 2023..."


_Однако здесь нет фильтров по контексту (подходит ли под обсуждение TON) и нет фильтра по датам, а также нет разделения комментариев и цитат. Это просто мой первый вариант парсинга, оставила просто как дополнение_

### Ссылка

**Ссылка на файлы:** [Облако mail.ru](https://cloud.mail.ru/public/D1SK/jBPhM872G) _(действительна 1 год)_ \
Здесь полученные таблицы + скаченные с Investing котировки